<a href="https://colab.research.google.com/github/yamkas/Aspect-Based-Sentiment-Analysis-On-Hotel-Reviews/blob/main/Web%20Scraping%20Google%20Maps%20With%20SerAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## instal Modul Python Google Search Results

In [1]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=e7b208f00ce09db17371e9f7c7024d82f8cd671ae9d2d1c4fa9f543b8dba2825
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


## Import Library

In [2]:
import pandas as pd
import numpy as np

#set warning
import warnings
warnings.filterwarnings('ignore')


pd.pandas.set_option('display.max_columns', None)

In [3]:
from serpapi import GoogleSearch
from urllib.parse import *

## Scrapping Local Result

Mengambil data 1 objek hotel dprima yang memiliki > 100 review

In [16]:
params = {
  "engine": "google_maps",
  "q": "hotel", # objek yang dicari
  "hl" : "id",
  "ll": "@-6.17000,106.63752,14.84z", #koordinat tempat sekitar
  "type": "search",
  "api_key": "d29a996fcdf9040a28e264ae355b50497bcf13bb5dce31f02b642756d1a36d08"
}

search = GoogleSearch(params)

lIdx = 0 # indeks yang ditampilkan dari data
lSum = 1 # Batas jumlah data yang ditampilkan
local_results = []

while lIdx  <= lSum:
  results = search.get_dict()

  for Result in results["local_results"]:
    lIdx += 1

    if lIdx <= lSum:

      #seleksi data (jangan ambil data jika jumlah review < 200 )
      if Result["reviews"] < 200:
        lIdx += 1
        continue

      # Append data sesuai dengan batas jumlah yang sudah ditentukan
      local_results.append({'Nama': Result["title"],
                           'data_id': Result["data_id"],
                           'total_reviews': Result["reviews"]});

  #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
  if "next" in results.get("serpapi_pagination", {}):
    # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
    search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

  else:
    break

In [5]:
params = {
  "engine": "google_maps",
  "q": "hotel", # objek yang dicari
  "hl" : "en",
  "ll": "@-6.17000,106.63752,14.84z", #koordinat tempat sekitar
  "type": "search",
  "api_key": "1359f542772592209a0680c2fa4bee6105e7782b301d630e66b2bf8fd93e8f40"
}

search = GoogleSearch(params)

lIdx = 0 # indeks yang ditampilkan dari data
lSum = 1 # Batas jumlah data yang ditampilkan
local_results = []

while lIdx  <= lSum:
  results = search.get_dict()

  for Result in results["local_results"]:
    lIdx += 1

    if lIdx <= lSum:

      #seleksi data (jangan ambil data jika jumlah review < 200 )
      if Result["reviews"] < 200:
        lIdx += 1
        continue

      # Append data sesuai dengan batas jumlah yang sudah ditentukan
      local_results.append({'Nama': Result["title"],
                           'data_id': Result["data_id"],
                           'total_reviews': Result["reviews"]});

  #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
  if "next" in results.get("serpapi_pagination", {}):
    # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
    search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

  else:
    break

In [6]:
for LR in local_results:
  print(LR)

{'Nama': "d'primahotel Tangerang", 'data_id': '0x2e69f8e661d3d2b1:0xd6bed8aaed31ba03', 'total_reviews': 8419}


## Scraping Ulasan
Mengambil 100 data dari masing-masing objek hotel sesuai dengan jumlah index yang diinginkan

In [18]:
data = {'nama_hotel':[],
        'nama_pelanggan':[],
        'rating':[],
        'ulasan':[]}

for LR in local_results:

  params = {
    "engine": "google_maps_reviews",
    "data_id": "",
    "api_key": "d29a996fcdf9040a28e264ae355b50497bcf13bb5dce31f02b642756d1a36d08",
    "hl": "id"
  }

  params["data_id"] = LR['data_id']

  search = GoogleSearch(params)

  lIdx = 0 # Angka indeks dari data
  lSum = 1000 # Batas jumlah data yang ditampilkan

  while lIdx <= lSum:
    results = search.get_dict()

    # Check if 'reviews' key exists in the results
    if "reviews" in results:
      for Result in results["reviews"]:
        lIdx += 1

        if lIdx <= lSum:
          # Append data sesuai dengan batas jumlah yang sudah ditentukan
          data['nama_hotel'].append(LR['Nama']);
          data['nama_pelanggan'].append(Result["user"]["name"]);
          data['rating'].append(Result["rating"]);
          data['ulasan'].append(Result.get("snippet", ""));

      #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
      if "next" in results.get("serpapi_pagination", {}):
        # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
        search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

      else:
        break
    else:
      # If 'reviews' key is not present, check for an error message and break the loop
      if "error" in results:
        print(f"Error fetching reviews for {LR['Nama']}: {results['error']}")
      break

review_original = data['ulasan']

In [8]:
data_2 = {'nama_hotel':[],
        'nama_pelanggan':[],
        'rating':[],
        'ulasan':[]}

for LR in local_results:

  params = {
    "engine": "google_maps_reviews",
    "data_id": "",
    "api_key": "1359f542772592209a0680c2fa4bee6105e7782b301d630e66b2bf8fd93e8f40",
    "hl": "en"
  }

  params["data_id"] = LR['data_id']

  search = GoogleSearch(params)

  lIdx = 0 # Angka indeks dari data
  lSum = 1000 # Batas jumlah data yang ditampilkan

  while lIdx <= lSum:
    results = search.get_dict()

    # Check if 'reviews' key exists in the results
    if "reviews" in results:
      for Result in results["reviews"]:
        lIdx += 1

        if lIdx <= lSum:
          # Append data sesuai dengan batas jumlah yang sudah ditentukan
          data_2['nama_hotel'].append(LR['Nama']);
          data_2['nama_pelanggan'].append(Result["user"]["name"]);
          data_2['rating'].append(Result["rating"]);
          data_2['ulasan'].append(Result.get("snippet", ""));

      #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
      if "next" in results.get("serpapi_pagination", {}):
        # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
        search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

      else:
        break
    else:
      # If 'reviews' key is not present, check for an error message and break the loop
      if "error" in results:
        print(f"Error fetching reviews for {LR['Nama']}: {results['error']}")
      break

review_original_2 = data_2['ulasan']

Error fetching reviews for d'primahotel Tangerang: Your account has run out of searches.


## Export Data Scrapping

In [19]:
import os

# Buat direktori jika belum ada
os.makedirs("File CSV", exist_ok=True)

dfScrap = pd.DataFrame(data)
dfScrap.to_csv('File CSV/ReviewHotelKotaTangerang_1.csv', index = False)

In [9]:
import os

# Buat direktori jika belum ada
os.makedirs("File CSV", exist_ok=True)

dfScrap_2 = pd.DataFrame(data_2)
dfScrap_2.to_csv('File CSV/ReviewHotelKotaTangerang_2.csv', index = False)

## Export Hasil ke CSV

In [21]:
dfScrap

,nama_hotel,nama_pelanggan,rating,ulasan
0,d'primahotel Tangerang,Lucky Widyanty,5.0,Menginap 6 hari 5 malam di D'prima Hotel. Kama...
1,d'primahotel Tangerang,Rita Rita,5.0,Pertama kalinya nginep di sini krn pas dapet h...
2,d'primahotel Tangerang,Kasa Aditya,4.0,27 Maret 2025\n\nDari bandara kami sudah dijem...
3,d'primahotel Tangerang,Choiril Chavid,5.0,"saya dari group jamaah haji Malika Tour, mengi..."
4,d'primahotel Tangerang,Robert Aritonang,5.0,"Lokasi ga jauh dari tol. Pelayanannya baik, st..."
...,...,...,...,...
995,d'primahotel Tangerang,Milo,4.0,Strategis di daerah Taman Royal berdekatan den...
996,d'primahotel Tangerang,Tri Marga Prastianggi,4.0,"Bagus,hnya ad kurang bgus yaitu pintu masuknya..."
997,d'primahotel Tangerang,Septian Bayu Nugraha,5.0,Overall oke. Seluruh staff2 nya rasa2nya ramah...
998,d'primahotel Tangerang,Ramdan Gsi,5.0,Nyaman


In [10]:
dfScrap_2

,nama_hotel,nama_pelanggan,rating,ulasan
0,d'primahotel Tangerang,Kadir SEREN,5.0,Amazing location. So easy to find. Room was so...
1,d'primahotel Tangerang,Catherine O'Callaghan,5.0,Nice hotel with excellent customer service! In...
2,d'primahotel Tangerang,Tekinci Biril (Türkçe Konuşan Dünya),5.0,So close to airport and room was big and styli...
3,d'primahotel Tangerang,novi Reksanto,5.0,Like to stay in this hotel. The food is delici...
4,d'primahotel Tangerang,Denial Aziz,5.0,"""I got a fantastic stay experience! The room w..."
...,...,...,...,...
983,d'primahotel Tangerang,Iwan Ridwan,5.0,"Deket Alfa MIDI, hanya 25 menit ke lubana seng..."
984,d'primahotel Tangerang,Trip.com Member,5.0,地理位置非常棒，门口打车3分钟内到，交通非常方便，冲这一点给五星好评
985,d'primahotel Tangerang,idham K,3.0,Saya menginap 1 malam tgl 18-19 Nov 2023\n\nS...
986,d'primahotel Tangerang,Trip.com Member,5.0,hotel yang nyaman dan lokasi strategis
